# Hill Cipher

Hill cipher is a cipher technique to encrypt plain text using key as a matrix.
Assume the key matrix is K and we need to encrypt plain text P, then cipher text C would be -
C = P x K
We must make sure that dimensions of P and K are compatible for multiplication and inverse of K should exist.

We would take the key K's dimension as 4x4. So, dimension of C and P must be n x 4.

## Implementation

Firstly we define an encoding function that will be used to take plaintext and convert it to a 26 character encoding. (By converting all letters to upper case and discarding all remaining characters). We also add padding character Z to the end of the string to make the length a multiple of 4 (if needed)

Also, let M = 4 be the dimension of the square key matrix.

In [1]:
M = 4

In [2]:
def encode(string):
    result = ''
    for letter in string:
        if letter.isalpha():
            result += letter.upper()
    padding_length = (M - len(result)) % M
    return result + 'Z' * padding_length

def decode(string):
    return string.rstrip('Z')

Lets declare a plain text that we would need to encrypt.

In [3]:
P = 'This is another plain text.'

Encoding this string, we get -

In [4]:
T = encode(P)
print(T)

THISISANOTHERPLAINTEXTZZ


Since we will be only having 26 characters, we declare Zp as closed ring of 26 integers.

In [5]:
Zp = Integers(26)
print(Zp)

Ring of integers modulo 26


We now declare a key matrix for the encryption.

In [6]:
elements = [[randint(0, 25) for _ in range(M)] for __ in range(M)]
key_m = matrix(Zp, M, M, elements)
while True:
    elements = [[randint(0, 25) for _ in range(M)] for __ in range(M)]
    key_m = matrix(Zp, M, M, elements)
    try:
        key_m.inverse()
        break
    except Exception as e:
        continue

# function to convert matrix to text
def matrix_to_text(mat):
    text = ''
    for row in mat:
        for e in row:
            text += chr(int(e) + ord('A'))
    return text

# function to convert text to matrix with M
def text_to_matrix(string):
    text_elements = []
    for i in range(0, len(string), M):
        text_elements.append([ord(x) - ord('A') for x in list(string[i:i+M])])
    return matrix(Zp, len(string) / M, M, text_elements)
    
print(key_m)
key = matrix_to_text(key_m)
print(key)

[23  7 11 23]
[23  4 20 21]
[23  1  2 18]
[10 22 21  4]
XHLXXEUVXBCSKWVE


Using the definition of hill cipher, we can encrypt each character of text using -

C = P x K

We must also add padding characters 'Z' at the end of the plain text to make the length a multiple of M(=4) and then convert the text into a matrix.

In [7]:
def hillcipher(text, cipher_key):
    cipher = ''
    
    key_mat = text_to_matrix(cipher_key)
    
    text_matrix = text_to_matrix(text)
    cipher_matrix = text_matrix * key_mat
    
    cipher = matrix_to_text(cipher_matrix)
    
    return cipher
    

def hilldecipher(cipher_text, cipher_key):
    text = ''
    
    cipher_matrix = text_to_matrix(cipher_text)
    key_mat = text_to_matrix(cipher_key)
    
    text_matrix = cipher_matrix * key_mat.inverse()
    
    text = matrix_to_text(text_matrix)
    return text

Now we test the cipher and decipher algorithms by encrypting and decrypting the text P

In [8]:
T = encode(P)
C = hillcipher(T, key)
D = hilldecipher(C, key)
D_stripped = decode(D)
print(f'Given text - "{P}"')
print(f'Encoded - {T}')
print(f'Key -\n{key}')
print(f'Cipher text - {C}')
print(f'Decipher text - {D}')
print(f'Stripped text - {D_stripped}')

Given text - "This is another plain text."
Encoded - THISISANOTHERPLAINTEXTZZ
Key -
XHLXXEUVXBCSKWVE
Cipher text - ATPUAYTQYJIFBIPUYHCJXGMW
Decipher text - THISISANOTHERPLAINTEXTZZ
Stripped text - THISISANOTHERPLAINTEXT


## Test Against Builtin Cipher

Now, we can test the result against the built in Hill Cipher in sagemath.

In [9]:
A = HillCryptosystem(AlphabeticStrings(), 4)
E = A.encoding(encode(P))
print(f'Text - {P}')
print(f'Encoded - {E}')
print(f'Key -\b{key}')
C_test = A.enciphering(text_to_matrix(key), E)
D_test = A.deciphering(text_to_matrix(key), C_test)

# convert to python string
C_test = str(C_test)
D_test = str(D_test)
D_test_stripped = decode(D_test)

print(f'Cipher text - {C_test}')
print(f'Decipher text - {D_test}')
print(f'Stripped text - {D_test_stripped}')

Text - This is another plain text.
Encoded - THISISANOTHERPLAINTEXTZZ
Key -XHLXXEUVXBCSKWVE
Cipher text - ATPUAYTQYJIFBIPUYHCJXGMW
Decipher text - THISISANOTHERPLAINTEXTZZ
Stripped text - THISISANOTHERPLAINTEXT


Comparing the built in cipher result with our implementation -

In [10]:
print('Results \t Implementation \t Built-in\n')
print(f'Cipher Text \t {C} \t {C_test}')
print(f'Decipher Text \t {D} \t {D_test}\n')
if C_test == C and D_test == D:
    print('Implementation is CORRECT')
else:
    print('Implementatiokn is INCORRECT')

Results 	 Implementation 	 Built-in

Cipher Text 	 ATPUAYTQYJIFBIPUYHCJXGMW 	 ATPUAYTQYJIFBIPUYHCJXGMW
Decipher Text 	 THISISANOTHERPLAINTEXTZZ 	 THISISANOTHERPLAINTEXTZZ

Implementation is CORRECT


## Cryptoanalysis

Known plain text attack can be done on hill cipher. Simple matrix arithematic can be used to obtain the key if plain text and cipher text pair are known along with key size.

We know that -
C = P x K

Therefore,
K = (P^-1) * C

Also, the plain text matrix should be invertible. So, we take an invertible 4x4 matrix and convert it to plain text.

In [11]:
P_elems = [[randint(0, 25) for _ in range(M)] for __ in range(M)]
P_mat = matrix(Zp, M, M, P_elems)
while True:
    P_elems = [[randint(0, 25) for _ in range(M)] for __ in range(M)]
    P_mat = matrix(Zp, M, M, P_elems)
    try:
        P_mat.inverse()
        break
    except Exception as e:
        continue
P_attack = matrix_to_text(P_mat)
print('Taking plain text as -', P_attack)

Taking plain text as - NRCBJDRRJDSOZOVK


We can now get the cipher text using the cipher function. In real world scenario, when using known plain text attack, we only have access to the encryption mechanism and not the key itself.

Here, we assume that the function `hillcipher` is the mechanism we have access to but not the key itself.

In [12]:
C_attack = hillcipher(P_attack, key)
print('Cipher text received -', C_attack)

Cipher text received - SBOUFYEUYLVAYEBN


Now, we can convert the cipher text into cipher matrix and then calculate the key.

In [13]:
C_mat = text_to_matrix(C_attack)
# calculate the key
K_predict = P_mat.inverse() * C_mat
K_predict_text = matrix_to_text(K_predict)
print('Key calculated - ', K_predict_text)

Key calculated -  XHLXXEUVXBCSKWVE


Now we use the cracked key to decipher the cipher text.

In [18]:
text_cracked = hilldecipher(C, K_predict_text)
print('Cracked text -', text_cracked)

Cracked text - THISISANOTHERPLAINTEXTZZ


If we compare the cracked text with the original plain text, we can check if the attack was successful.

In [19]:
if encode(P) == text_cracked:
    print('Encryption was broken')
else:
    print('Encryption couldn\'t be broken')

Encryption was broken


Hence, known plain text attack was successful on Hill Cipher.